In [31]:
%run BGgeothermal2.ipynb
import pandas as pd
from iapws import IAPWS97

ws = pd.read_excel("bom.xls", sheetname=2, parse_cols="A,H:T")
#print(ws.head())
owhp = pd.read_excel("bom.xls", sheetname=4)

svlist = {}
svwell = {}
data = {}
finalinfo = {}

#print (svwell)
def sep_steam_frac(twophaseenthalpy, separatorpressure):
    sep_steam = IAPWS97(P=separatorpressure, x=1)
    sep_liquid = IAPWS97(P=separatorpressure, x=0)
    return (twophaseenthalpy-sep_liquid.h)/(sep_steam.h-sep_liquid.h)


for eachindex in range(len(ws.index)):
    #create a class for each row; output curve equations
    wellname = ws.iloc[eachindex,0]
    mf1, mf2, mf3, mf4, mf5, mf6, h1, h2, h3, h4, h5, sep_press, vessel = map(lambda x: ws.iloc[eachindex, x], range(1,14))
    well_headpressure = owhp.iloc[eachindex,1]

    welldata = WellClass(well_headpressure, mf1, mf2, mf3, mf4, mf5, mf6, h1, h2, h3, h4, h5)#, sep_press, vessel)

    #create a dictionary of wellname: [MF, H]
    data[wellname] = [welldata.massflow(),welldata.enthalpy()]#,welldata.sep_steam_frac()]

    #create a dictionary of sep_vessel and the wells that supply it
    if vessel not in svwell.keys():
        #print ("not in list; ", wellname, vessel)
        svwell[vessel] = [wellname]
        #print (svwell)
    else:
        templist = svwell[vessel]
        templist.append(wellname)
        svwell[vessel] = templist


ws2 = pd.read_excel("bom.xls", sheetname = 3)
#print (ws2.head())
chemlist = {}

for i in range(len(ws2.index)):
    svlist[ws2.iloc[i, 0]] = VesselClass(ws2.iloc[i, 1]).sepP

ws3 = pd.read_excel("bom.xls", sheetname="ChemistryData")

for i in range(len(ws3.index)):
    # the format chemlist["PAL12D"] = [Cl,SiO2, CO2, H2S]"
    chemlist[ws3.iloc[i,0]] = [ws3.iloc[i,1],ws3.iloc[i,2],ws3.iloc[i,3],ws3.iloc[i,4]]

for eachvessel in svwell.keys():
    welllist = svwell[eachvessel]
    totalSF = 0
    totalBF = 0
    wellCO2 = 0
    wellH2S = 0
    totalMF = 0

    for eachwell in welllist:
        themassflow = data[eachwell][0]
        theenthalpy = data[eachwell][1]
        thesteamfraction = sep_steam_frac(theenthalpy, svlist[eachvessel])
        thesteamflow = thesteamfraction * themassflow
        thebrineflow = themassflow - thesteamflow
        wellCO2 += themassflow * chemlist[eachwell][2]
        wellH2S += themassflow * chemlist[eachwell][3]
        totalMF += themassflow
        totalSF += thesteamflow
        totalBF += thebrineflow

    finalinfo[eachvessel] = [totalSF, totalBF, totalMF, wellCO2/totalMF, wellH2S/totalMF, svlist[eachvessel]]

SFsum = 0
BFsum = 0
sv_out = {}
for eachvessel in svwell.keys():
    SFsum += finalinfo[eachvessel][0]
    BFsum += finalinfo[eachvessel][1]
    print(eachvessel, finalinfo[eachvessel])
    sv_out[eachvessel] = finalinfo[eachvessel]

print(sv_out['sv4034'])

svout_df = pd.DataFrame(data=sv_out)
svout_df = svout_df.transpose()
svout_df.columns = ['SF', 'WF', 'MF', 'CO2', 'H2S', 'SVP']

#print (chemlist)
writer = pd.ExcelWriter('sv_out.xls')
svout_df.to_excel(writer,'sv_out')
writer.save()
svout_df.head()

sv4034 [51.192737605342757, 85.158169841699859, 136.3509074470426, 149.19762292874387, 7.7078908029957551, 0.88]
sv3034 [26.81158408030182, 73.469956179751549, 100.28154026005336, 259.09136764596946, 7.977037736842953, 0.88]
sv5080 [33.506430763776635, 69.744218697997383, 103.25064946177403, 178.52223904705698, 3.8656473871485346, 0.78000000000000003]
sv3056 [39.305286683780075, 54.497223681549571, 93.802510365329653, 324.47377539051757, 7.6837532153323487, 0.88]
sv3012 [50.643663932310723, 65.458811118620417, 116.10247505093113, 515.89876745388267, 15.928105070020774, 0.88]
sv4050 [49.86740570754867, 126.15466858072372, 176.02207428827239, 193.13148497044349, 8.3692980402880774, 0.88]
sv4012 [42.277995894078465, 124.66901866648072, 166.94701456055918, 113.77082919256443, 6.1958576866875132, 0.88]
[51.192737605342757, 85.158169841699859, 136.3509074470426, 149.19762292874387, 7.7078908029957551, 0.88]


,SF,WF,MF,CO2,H2S,SVP
sv3012,50.643664,65.458811,116.102475,515.898767,15.928105,0.88
sv3034,26.811584,73.469956,100.281540,259.091368,7.977038,0.88
sv3056,39.305287,54.497224,93.802510,324.473775,7.683753,0.88
sv4012,42.277996,124.669019,166.947015,113.770829,6.195858,0.88
sv4034,51.192738,85.158170,136.350907,149.197623,7.707891,0.88
